In [1]:
import os
import napari
from skimage.io import imread
import tifffile
import numpy as np
import zarr
from skimage.util import invert

from scipy import ndimage as ndi
import pandas as pd
from skimage.filters import threshold_otsu
from skimage.exposure import rescale_intensity
from skimage.morphology import ball, binary_opening
from skimage.measure import label, regionprops

### Helper functions

In [2]:
def erase_small(labels, min_size):
    unique, counts = np.unique(labels, return_counts=True)
    for i in range(len(unique)):
        if counts[i]<min_size:
            labels[labels==unique[i]]=0
    return labels

    #remove_background
def remove_backg(img, sig1=1,sig2=15):
    ('removing background from immuno channel')
    #img = img.astype('int16')
    img_sig =ndi.gaussian_filter(img, (sig1, sig1,sig1))
    img_bcg = ndi.gaussian_filter(img, (sig2,sig2,sig2))
    cleaned = img_sig - img_bcg
    cleaned[cleaned<0]=0
    return cleaned

### 1. Load raw data

In [3]:
file_name = "../../Data/LICONN_gephyrin_test.tif"
data = tifffile.imread(file_name)

### 2. Detect gephyrin 

In [21]:
#select the channel
gephyrin = data[2,:,:,:]

#rescale intensity
img = gephyrin.astype('float32')
mi = 101
ma = np.percentile(img, 99.99)
img = rescale_intensity(img, in_range=(mi, ma))

#clean background
sig1=1
sig2=10
img_clean =remove_backg(img,sig1,sig2)

#threshold and label
threshold =  threshold_otsu(img_clean)
mask = img_clean > threshold
mask = binary_opening(mask,ball(4))
labels = label(mask)
labels_gephyrin = erase_small(labels, 150)

#get coordinates
rp = regionprops(labels_gephyrin)
gephyrin_coords = []
for r in rp:
    gephyrin_coords.append(np.array(r.centroid))

image cleanded
image threholded
image labeled


'\nfile = open(\'gephyrin_coords.txt\',\'w\')\nfor item in gephyrin_coords:\n\tfile.write(str(item)+"\n")\nfile.close()\n'

### 3. Visualize results in napari

In [23]:
viewer = napari.Viewer()
viewer.add_image(data[0,:,:,:],colormap='gray')
viewer.add_image(gephyrin,colormap = 'yellow', blending = 'additive')
viewer.add_points(np.stack(gephyrin_coords).astype('uint32'), size=10, face_color = 'red')

<Points layer 'Points' at 0x24c3e7d2c40>